In [ ]:
import toml
import pygsheets
from tqdm.auto import tqdm
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import benchlingapi
import sqlalchemy
from sqlalchemy import select, text
import xmltodict
from lxml import etree

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api.benchling as bapi
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.io as cio
import paulssonlab.cloning.sequence as sequence
import paulssonlab.api.geneious as geneious
import paulssonlab.api.geneious_db as g

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
bench_session = benchlingapi.Session(config["benchling"]["api_key"])
benchling_folder = bapi.get_project_root(bench_session, config["benchling"]["project"])

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"], benchling_folder)

# Config

In [ ]:
lib_parts = reg[("LIB", "parts")]
olib_oligos = reg[("oLIB", "oligos")]
plib_maps = reg[("pLIB", "maps")]
part_types = reg[("LIB", "parts", "Part types")]

# Client

In [ ]:
client = geneious.GeneiousClient(**config["geneious_test_local"])

In [ ]:
client.subfolder[
    ("foo", "bar")
]  # -> GeneiousClient with root=("foo", "bar"); shares GeneiousClientState dataclass with original
client.subfolder[("foo", "bar")].items()  # a way of iterating a subfolder
client["rootseq"]
client[("foo", "bar", "childseq")]  # -> DsSeqRecord/Primer
client.raw["rootseq"]  # -> ORM object?
client.raw["rootseq"]  # -> hhh

# Test

## DB

In [ ]:
db_url = sqlalchemy.engine.URL.create(**config["geneious_test_local"])
engine = sqlalchemy.create_engine(db_url, future=True, echo=True)
Session = sqlalchemy.orm.sessionmaker(bind=engine, future=True)

## DB test

In [ ]:
session = Session()

In [ ]:
x = session.execute(select(g.Folder).where(g.Folder.name == "plasmids")).one()[0]

In [ ]:
x.annotated_documents

In [ ]:
y = x.annotated_documents[0]

In [ ]:
y.additional_document_xml

In [ ]:
session.execute("SELECT max(id) FROM folder").one()

In [ ]:
session.execute("SELECT current_timestamp()").one()

In [ ]:
session.execute("SELECT document_urn, element_key FROM additional_document_xml").all()

In [ ]:
import datetime

In [ ]:
datetime.datetime.now()

In [ ]:
# start another geneious client (with another postgres user), add folder, see what triggers select from next_table_id

# check cadence of select, see if the big join produces expected output
# (difference between python folders and geneious folders?)

# try adding new folder, changing modified of parent folder, see if geneious selects from next_table_id

In [ ]:
# 2022-02-21 17:26:39.479 EST,"newuser","geneious_test1",23897,"127.0.0.1:64955",62140436.5d59,1580,"SELECT",2022-02-21 16:29:26 EST,4/1573,0,LOG,00000,"execute S_4: SELECT A.id,A.name,g_group_id,A.modified,document_urn,parent_folder_id,visible, C.xml AS folderColourXml, (CASE WHEN C.modified IS NOT NULL THEN C.modified ELSE CURRENT_TIMESTAMP END) AS folderColourModified FROM folder A LEFT OUTER JOIN folder_view B ON A.id = B.folder_id LEFT OUTER JOIN special_element C ON A.id = C.folder_id AND C.name = 'folderColor' WHERE A.modified > $1 AND A.modified < $2 AND id NOT IN (select hidden_folder_id from hidden_folder_to_user WHERE user_id != $3)","parameters: $1 = '2022-02-21 17:00:43.453', $2 = '2022-02-21 17:26:39.479', $3 = '1'",,,,,,,,"PostgreSQL JDBC Driver","client backend",,0

In [ ]:
for idx in range(40):
    with Session.begin() as session:
        group = session.execute(
            select(g.GGroup).where(g.GGroup.name == "Everybody")
        ).one()[0]
        print("group", group)
        parent = session.execute(select(g.Folder).where(g.Folder.name == "xxx5")).one()[
            0
        ]
        print(parent)
        new_name = f"sub{idx}"
        new_folder = g.Folder(name=new_name, g_group=group)
        new_folder.parent_folder = parent
        parent.modified = text("current_timestamp")  # datetime.datetime.utcnow()
        session.add(new_folder)
        # parent.children[new_name] = new_folder
        # session.execute(update(sql.functions.max(g.Table.id)))
        # session.execute("UPDATE next_table_id SET next_id = (SELECT max(id) FROM folder) WHERE table_name = 'folder'")

In [ ]:
with Session.begin() as session:
    q = select(
        g.Folder
    )  # .where(g.Folder.name == "bot4")#.where(g.Folder.visible == True)
    res = session.execute(q).all()
    print("\n".join(map(str, res)))

In [ ]:
with Session.begin() as session:
    q = select(g.NextTableId)
    res = session.execute(q).all()
    print("\n".join(map(str, res)))

In [ ]:
with Session.begin() as session:
    res = session.execute(text("SELECT * from folder;")).all()
    print("\n".join(map(str, res)))

In [ ]:
with Session.begin() as session:
    q = select(g.NextTableId)
    res = session.execute(q).all()
    print("\n".join(map(str, res)))

In [ ]:
with Session.begin() as session:
    q = select(g.Folder)
    res = session.execute(q).all()
    print("\n".join(map(str, res)))

## XML

In [ ]:
seq = plib_maps["pLIB112"]

In [ ]:
from xmldiff.main import diff_texts

In [ ]:
output = etree.tostring(geneious.dump_geneious(seq4), pretty_print=True)

In [ ]:
x = etree.fromstring(doc4)

In [ ]:
geneious.load_geneious(x)

In [ ]:
seq4.annotations

In [ ]:
seq4.annotations["references"][0].__class__

In [ ]:
geneious.loads_geneious(doc4).annotations

## Round-trip: seq->geneious-seq

## Round-trip: geneious->seq->geneious

## Old

In [ ]:
for name, t in metadata.tables.items():
    print(name)
    q = select([t]).limit(20)
    rows = conn.execute(q).all()
    for row in rows:
        print(row)
    print()

In [ ]:
q = select(g.AnnotatedDocument)
rows = conn.execute(q).all()

In [ ]:
doc0 = rows[0]["plugin_document_xml"]

In [ ]:
doc4 = rows[-1]["plugin_document_xml"]

In [ ]:
import os

In [ ]:
seq4 = cio.read_file(os.path.expanduser("/Users/jacob/Downloads/_plasmids/genbank2.gb"))

In [ ]:
seq4

In [ ]:
print(doc4)

In [ ]:
doc2 = rows[1]["plugin_document_xml"]

In [ ]:
rows[5]["xml_element"]

In [ ]:
docs = [xmltodict.parse(row["document_xml"], dict_constructor=dict) for row in rows]

In [ ]:
BAD_CLASSES = [
    # "com.biomatters.geneious.publicapi.implementations.sequence.OligoSequenceDocument",
    "com.biomatters.geneious.publicapi.documents.OperationRecordDocument",
]
good_docs = [d for d in docs if d["document"]["@class"] not in BAD_CLASSES]

In [ ]:
set(d["document"]["@class"] for d in good_docs)

In [ ]:
rows[2]["document_xml"]

In [ ]:
x = xmltodict.parse(rows[2]["plugin_document_xml"], dict_constructor=dict)

In [ ]:
x